Imports que importam :) 

In [30]:
import numpy as np
import math

import pandas

In [31]:
from random import *
cutpart = lambda x: (int((x+1) * np.random.normal(scale=0.2,loc=0.5)) )% x

In [32]:
va = [x for x in range(1,11)]
vb = [x for x in range(10,0,-1)]
print("va:\n{}\nvb:\n{}".format(va,vb))

va:
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
vb:
[10, 9, 8, 7, 6, 5, 4, 3, 2, 1]


<h1>Genetic Algorithm for QAP solving()</h1>

In [33]:
def crossover(va,vb):
    cut = cutpart(len(va))
    print("cut: {}".format(cut))
    new_va = va[:cut]+vb[cut:]
    new_vb = vb[:cut]+va[cut:]
    print("new va:\n{}\nnew vb:\n{}".format(new_va,new_vb))
    #encontrar repetidos (x) e suas repectivas posicoes (i)
    repeated_a = [(x,i) for x,i in zip(new_va,range(len(va))) if new_va.count(x) > 1]
    repeated_b = [(x,i) for x,i in zip(new_vb,range(len(vb))) if new_vb.count(x) > 1]
    rn = int(len(repeated_a)/2)
    print("rn:{}".format(rn))
    for i in range(rn):
        print("switch {} / {}".format(i,rn))
        new_va[repeated_a[i][1]] = repeated_b[i+ (0 if cut >5 else rn)][0]	#se trocar sempre o primeiro 
        new_vb[repeated_b[i][1]] = repeated_a[i+ (0 if cut >5 else rn)][0]	#repetido cortes acima de 5 tendem
    print("repeated a: {}".format(repeated_a[:int(len(repeated_a))]))		#a repertir os valores dos pais
    print("repeated b: {}".format(repeated_b[:int(len(repeated_b))]))
    return(new_va,new_vb)

In [34]:
new_va,new_vb = crossover(va,vb)#[1,5,3,4,6,10,7,9,8,2])
print("new va:\n{}\nnew vb:\n{}".format(new_va,new_vb))

cut: 4
new va:
[1, 2, 3, 4, 6, 5, 4, 3, 2, 1]
new vb:
[10, 9, 8, 7, 5, 6, 7, 8, 9, 10]
rn:4
switch 0 / 4
switch 1 / 4
switch 2 / 4
switch 3 / 4
repeated a: [(1, 0), (2, 1), (3, 2), (4, 3), (4, 6), (3, 7), (2, 8), (1, 9)]
repeated b: [(10, 0), (9, 1), (8, 2), (7, 3), (7, 6), (8, 7), (9, 8), (10, 9)]
new va:
[7, 8, 9, 10, 6, 5, 4, 3, 2, 1]
new vb:
[4, 3, 2, 1, 5, 6, 7, 8, 9, 10]


In [35]:
def inverse(vec):
    inf = int(random()*len(vec))
    part = list(vec[inf:])
    part.reverse()
    return vec[:inf]+part

In [36]:
def swap (vec):
    n_vec = list(vec)
    p1 = int(random()*len(vec))
    p2 = int(random()*len(vec))
    v1 = vec[p1]
    v2 = vec[p2]
    print("swap {} and {}".format(v1,v2))
    n_vec[p1] = v2
    n_vec[p2] = v1
    return n_vec

In [37]:
def insertion(vec):
    inf = int(random()*len(vec))
    part = vec[inf:]
    part = part[1:] + part[:1]
    print(part)
    return vec[:inf]+part

In [38]:
def mutation(vec,p):#percentual
    if random() > p:
        return vec
    typ = random()
    if(typ < 1/3):
        print("inverse")
        vec = inverse(vec)
    elif(typ < 2/3):
        print("swap")
        vec = swap(vec)
    else:
        print("insertion")
        vec = insertion(vec)
    return vec
    
#mutation(va,0.7)
print("Original:")
print(va)
print("Inverse:")
print(inverse(va))
print("Swap:")
print(swap(va))
print("Insertion")
print(insertion(va))

Original:
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
Inverse:
[10, 9, 8, 7, 6, 5, 4, 3, 2, 1]
Swap:
swap 2 and 1
[2, 1, 3, 4, 5, 6, 7, 8, 9, 10]
Insertion
[3, 4, 5, 6, 7, 8, 9, 10, 2]
[1, 3, 4, 5, 6, 7, 8, 9, 10, 2]


In [39]:
mutation(va,0.7)

insertion
[5, 6, 7, 8, 9, 10, 4]


[1, 2, 3, 5, 6, 7, 8, 9, 10, 4]

<h1>Calculo de Coorelação:</h1>

In [40]:
def corr(x,y,p=False):
    # n
    n = len(x)
#Somatórios:
    #superiores
    xysum = np.dot(x,y) #soma dos x*y
    xsum = sum(x)
    ysum = sum(y)
    #inferiores
    #soma dos quadrados
    x2sum = np.dot(x,x) #multiplica entrada a entrada dos vetores x e ele mesmo e soma
    y2sum = sum([v*v for v in y]) #forma na unha de fazer o mesmo
    #quadrado das somas:
    xsum2 = xsum**2
    ysum2 = ysum**2
    #parte superior:
    sup = (xysum - xsum*ysum/n)
    #parte inferior:
    infe = math.sqrt ( (x2sum - xsum2/n) * (y2sum - ysum2/n) )
    corresult = sup/infe
    if(p):
        print("Taxa de correlação entre X e Y é:")
        print("{}\n".format(sup)+len(str(sup))*"-"+" =  {}".format(corresult)+"\n{}".format(infe))
    return corresult

<h2>Teste Da Correlação</h2>

In [41]:
gerax = lambda base,n,dif: [base + x*dif for x in range(n)] # modo preguiçoso de gerar n números espaçados em 0.5
#Carregar valores(teste):
x = gerax(2.5, 6,0.5)
y = [0.4,0.4,0.3,0.2,0.2,0.1]
corr(x,y,True)

Taxa de correlação entre X e Y é:
-0.5499999999999998
------------------- =  -0.9710083124552239
0.5664215155988811


-0.9710083124552239

In [42]:
#!/usr/bin/python

from pathlib import Path

directory = './nodes_links/'
allnodes = dict()
alllinks = dict()
names = list()
for path in Path(directory).iterdir():
    url = str(path)
    netname = path.name.split('_')[0]
    if(path.name.count('nodes')==1):
        p = allnodes
    else:
        p = alllinks
    try:
        item = pandas.read_csv(url)
        p.update( {netname : item })
    #    print(url+" lido")
    except:
        try:
            #mudar o encoding para os caracteres
            item = pandas.read_csv(url,encoding="iso-8859-1")
            p.update( {netname : item })
        #    print(url+" lido")
        except:
            print("!!! não foi possivel ler > "+name+" < !!!")
            pass
    if names.count(netname) == 0:
        names.append(netname) 


In [43]:
print(names)

netlinks = alllinks['portugal']
netnodes = allnodes['portugal']

['canarie', 'italy', 'vbns', 'loni', 'germany', 'geant2', 'arpanet', 'cesnet', 'newnet', 'rnpBrazil', 'pionier', 'spain', 'renater', 'lambdaRailUsa', 'metrona', 'arnes', 'bren', 'portugal', 'usaGde', 'sanet', 'nsfnet', 'austria', 'viaDatacenterNet', 'coxUsa', 'mzima', 'scteste', 'eon', 'memorexEurope', 'deutschTelecom', 'internet2Usa', 'OmnicomEurope']


In [44]:
#u = pandas.read_csv("./nodes_links/renater_links.csv")
a = netnodes['Id'].values.tolist()
print("nodes:\n{}".format(a))
print("links:\n{}".format(netlinks))

nodes:
['Alcácer_do_Sal', 'Aveiro', 'Beja', 'Braga', 'Bragança', 'Caldas_Rainha', 'Castelo_Branco', 'Coimbra', 'Elvas', 'Évora', 'Faro', 'Funchal', 'Guarda', 'Leiria', 'Lisboa', 'Ponta_Delgada', 'Portalegre', 'Portimão', 'Porto', 'Santarém', 'S_João_Madeira', 'Setúbal', 'Sines', 'Viana_do_Castelo', 'Vila_Real', 'Viseu']
links:
                From              To  Length  Capacity    Cost
0             Lisboa   Ponta_Delgada    1500         1  975000
1             Lisboa         Funchal    1050         1  682500
2      Ponta_Delgada         Funchal    1050         1  682500
3           Portimão         Funchal    1010         1  656500
4              Braga        Bragança     209         1  135850
5           Santarém      Portalegre     144         1   93600
6               Beja            Faro     140         1   91000
7           Portimão           Sines     139         1   90350
8           Santarém         Coimbra     136         1   88400
9           Bragança       Vila_Real     

In [45]:
#print(netlinks.loc[:, ['From','To']])
#import networkx as nx
#g = nx.DiGraph()
#nodes = [key for key in u.groupby("From").indices]
def monta_matriz(netnodes,netlinks):
    nodes = netnodes['Id'].values.tolist()
    matrix = np.matrix(np.zeros( (len(nodes),len(nodes)),dtype=np.int ) )
    for n in nodes:
        l = netlinks.loc[netlinks['From']==n,['To']].values
        indexes = [nodes.index(x) for x in l]
        if(len(indexes) > 0):
            matrix[nodes.index(n),indexes] = 1
            matrix[indexes,nodes.index(n)] = 1 #bidirecional
    return matrix
#l = u.loc[u['From']=="Boston",['To']]
#print(l.values)
#print(nodes.index("NewYork"))
#nodes.index("Boston")
#g.add_nodes_from(nodes)

matrix = monta_matriz(netnodes,netlinks)
print (matrix)

[[0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0]
 [0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 1 0 0]
 [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 1]
 [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
 [1 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
 [0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 1 0 0 0 1 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0]
 [0 1 0 1 

In [46]:
def calc_grau(matrix):
    soma=0
    for l in range(len(matrix)):
        for c in range(len(matrix)):
            soma += matrix[l,c]
    return soma/(len(matrix))

In [47]:
print("grau médio: {}".format(calc_grau(matrix)))

grau médio: 2.769230769230769


In [48]:
def aumentar_grau(mtx,inc_grau): #nodos = 6, p/incrementar 1 grau adicionamos 6 enlaces, para inc 0.5 add 3
    grau = calc_grau(mtx)
    mind = len(mtx)-1
    n_grau = grau
    while n_grau < mind and n_grau < grau + inc_grau:
        x,y=(None,None)
        while (x==None and y==None):
            x,y = (randint(0,mind), randint(0,mind))
            if x==y or mtx[x,y]==1:
                x,y=(None,None)
        mtx[x,y]=1
        mtx[y,x]=1 #bijetora...
        n_grau=calc_grau(mtx)

In [49]:
aumentar_grau(matrix,10)

In [50]:
print(matrix)

[[0 0 1 1 0 0 0 0 0 1 0 1 0 0 1 0 0 0 1 0 0 1 1 0 0 0]
 [0 0 0 0 0 1 0 1 1 0 1 0 1 1 0 1 0 0 1 1 1 1 1 0 0 1]
 [1 0 0 0 0 1 0 1 0 1 1 0 0 1 0 1 1 1 1 0 0 1 0 0 0 1]
 [1 0 0 0 1 0 0 0 1 1 1 0 1 0 1 1 0 0 1 0 1 1 1 1 0 0]
 [0 0 0 1 0 0 1 1 0 1 1 1 1 0 0 1 1 0 0 0 1 1 0 1 1 1]
 [0 1 1 0 0 0 0 0 1 1 0 0 0 1 1 1 0 1 0 1 0 1 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0 1 0 1 1 1 0 1 0 0 1 0 1 0 0 1 0]
 [0 1 1 0 1 0 0 0 0 1 1 1 0 1 0 0 1 0 1 1 1 0 1 1 1 1]
 [0 1 0 1 0 1 0 0 0 1 1 1 0 0 0 1 1 1 1 1 1 1 1 1 1 0]
 [1 0 1 1 1 1 0 1 1 0 0 1 1 1 1 1 0 0 0 0 0 0 0 0 1 0]
 [0 1 1 1 1 0 1 1 1 0 0 0 0 1 0 0 1 1 1 1 1 0 1 1 1 0]
 [1 0 0 0 1 0 0 1 1 1 0 0 1 0 1 1 1 1 1 0 0 1 0 1 1 0]
 [0 1 0 1 1 0 1 0 0 1 0 1 0 0 1 1 0 1 1 1 0 1 1 0 0 1]
 [0 1 1 0 0 1 1 1 0 1 1 0 0 0 1 1 0 0 0 1 0 0 0 0 1 0]
 [1 0 0 1 0 1 1 0 0 1 0 1 1 1 0 1 1 0 1 1 0 1 0 0 0 1]
 [0 1 1 1 1 1 0 0 1 1 0 1 1 1 1 0 0 1 1 1 0 1 1 0 1 1]
 [0 0 1 0 1 0 1 1 1 0 1 1 0 0 1 0 0 0 0 1 0 0 0 0 0 0]
 [0 0 1 0 0 1 0 0 1 0 1 1 1 0 0 1 0 0 0 0 1 0 1 1 1 0]
 [1 1 1 1 

In [51]:
print("grau médio: {}".format(calc_grau(matrix)))

grau médio: 12.76923076923077


In [52]:
aumentar_grau(matrix,100)

In [53]:
print("grau médio: {}".format(calc_grau(matrix)))

grau médio: 25.0


In [54]:
print(matrix)

[[0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]
 [1 1 1 1 

In [68]:
matrizes = dict()
for x in names:
    netlinks = alllinks[x]
    netnodes = allnodes[x]
    matriz = monta_matriz(netnodes,netlinks)
    matrizes.update({x:matriz})
    print("matriz {}:\n{}".format(x,matriz))

matriz canarie:
[[0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 1 0 1 0 1 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 1]
 [0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 1 0 1 1 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0 1 1 0 0 1 1 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 1]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1]
 [0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 1 0]]
matriz italy:
[[0 1 0 1 0 0 0 0 0 0 0 0 0 0]
 [1 0 1 1 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 1 1 0 0 0 0 0 0 0 0 0]
 [1 1 1 0 1 1 1 1 0 0 1 0 0 1]
 [0 0 1 1 0 1 0 0 0 0 0 0 0 0]
 [0 0 0 1 1 0 1 0 1 0 1 0 0 1]
 [0 

matriz metrona:
[[0 1 0 ... 0 0 0]
 [1 0 1 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 1 0]
 [0 0 0 ... 1 0 0]
 [0 0 0 ... 0 0 0]]
matriz arnes:
[[0 0 0 0 0 0 0 1 0 0 0 0 1 1 0 1 0]
 [0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0]
 [0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0]
 [1 1 0 0 1 0 0 0 1 0 0 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1]
 [1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0]]
matriz bren:
[[0 1 1 0 0 0 0 0 0 0]
 [1 0 0 0 0 1 0 0 0 0]
 [1 0 0 1 0 0 0 0 0 0]
 [0 0 1 0 1 0 0 0 0 1]
 [0 0 0 1 0 1 0 0 0 0]
 [0 1 0 0 1 0 0 1 0 0]
 [0 0 0 0 0 0 0 1 1 0]
 [0 0 0 0 0 1 1 0 0 0]
 [0 0 0 0 0 0 1 0 0 1]


 [1 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0]]
matriz internet2Usa:
[[0 1 1 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 1 0]
 [0 0 0 ... 1 0 1]
 [0 0 0 ... 0 1 0]]
matriz OmnicomEurope:
[[0 1 1 ... 0 0 1]
 [1 0 1 ... 0 0 0]
 [1 1 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 1]
 [1 0 0 ... 0 1 0]]


In [69]:
calc_grau(matrizes.get('cesnet'))

3.1666666666666665

In [70]:
print (matrizes.get('cesnet'))
print(len(matrizes.get('cesnet')))

[[0 1 1 1 0 0 0 0 0 1 1 1]
 [1 0 1 0 0 0 0 0 0 0 0 0]
 [1 1 0 1 0 0 0 0 0 0 0 0]
 [1 0 1 0 1 0 0 0 0 0 0 1]
 [0 0 0 1 0 1 1 1 0 0 0 0]
 [0 0 0 0 1 0 0 1 0 0 0 0]
 [0 0 0 0 1 0 0 1 0 0 0 0]
 [0 0 0 0 1 1 1 0 1 1 0 0]
 [0 0 0 0 0 0 0 1 0 1 0 0]
 [1 0 0 0 0 0 0 1 1 0 1 0]
 [1 0 0 0 0 0 0 0 0 1 0 0]
 [1 0 0 1 0 0 0 0 0 0 0 0]]
12


In [71]:
mtx = matrizes.get('cesnet')
aumentar_grau(mtx,4)
print (mtx)
print (calc_grau(mtx))

[[0 1 1 1 0 1 1 0 0 1 1 1]
 [1 0 1 0 1 1 0 1 1 1 1 0]
 [1 1 0 1 0 0 1 1 0 0 1 1]
 [1 0 1 0 1 1 0 0 0 1 0 1]
 [0 1 0 1 0 1 1 1 0 0 0 1]
 [1 1 0 1 1 0 1 1 0 1 1 0]
 [1 0 1 0 1 1 0 1 1 0 0 0]
 [0 1 1 0 1 1 1 0 1 1 1 0]
 [0 1 0 0 0 0 1 1 0 1 1 1]
 [1 1 0 1 0 1 0 1 1 0 1 1]
 [1 1 1 0 0 1 0 1 1 1 0 1]
 [1 0 1 1 1 0 0 0 1 1 1 0]]
7.166666666666667
